## Reference :-
https://scikit-learn.org/stable/auto_examples/text/plot_document_clustering.html#sphx-glr-auto-examples-text-plot-document-clustering-py

## Loading The the Text Data

In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

In [ ]:
newsgroups_train = fetch_20newsgroups(subset='train')
categories = [
    'alt.atheism',
    'comp.graphics',
    'comp.sys.ibm.pc.hardware',
    'comp.sys.mac.hardware',
    'misc.forsale',
    'rec.autos',
    'rec.motorcycles',
    'sci.electronics',
    'talk.politics.guns',
    'talk.politics.mideast',
    'talk.religion.misc',
]

In [ ]:
datasets = fetch_20newsgroups(
    remove = ('headers','footers','quotes'),
    subset = 'all',
    categories = categories,
    shuffle=True,
    random_state=42,
)

labels = datasets.target
unique_labels, category_size = np.unique(labels, return_counts=True)
true_k = unique_labels.shape[0]
print(f"There are {len(datasets.data)} documents in {true_k} categories")

There are 10140 documents in 11 categories


## Quantifying the quality of clustering results.

## Eventhough Clustering algorithms are unsupervised learning methods. Since we have class labels for this specific dataset, it is possible to use evaluation metrics that leverage this “supervised” ground truth information to quantify the quality of the resulting clusters. Examples of such metrics are the following:

1. Homogeneity, which quantifies how much clusters contain only members of a single class;

2. Completeness, which quantifies how much members of a given class are assigned to the same clusters;

3. V-measure, the harmonic mean of completeness and homogeneity;

4. Rand-Index, which measures how frequently pairs of data points are grouped consistently according to the result of the clustering algorithm and the ground truth class assignment;

5. Adjusted Rand-Index, a chance-adjusted Rand-Index such that random cluster assignment have an ARI of 0.0 in expectation.

In [ ]:
from collections import defaultdict
from time import time

from sklearn import metrics

evaluations = []        # Empty list to store estimator name and training time
evaluations_std = []    # Empty list to store estimator name and std dev of training time


def fit_and_evaluate(km, X, name=None, n_runs=5):
    name = km.__class__.__name__ if name is None else name

    train_times = []
    scores = defaultdict(list)
    for seed in range(n_runs):
        km.set_params(random_state=seed)
        t0 = time()
        km.fit(X)
        train_times.append(time() - t0)
        scores["Homogeneity"].append(metrics.homogeneity_score(labels, km.labels_))
        scores["Completeness"].append(metrics.completeness_score(labels, km.labels_))
        scores["V-measure"].append(metrics.v_measure_score(labels, km.labels_))
        scores["Adjusted Rand-Index"].append(
            metrics.adjusted_rand_score(labels, km.labels_)
        )
        scores["Silhouette Coefficient"].append(
            metrics.silhouette_score(X, km.labels_, sample_size=2000)
        )
    train_times = np.asarray(train_times)

    print(f"clustering done in {train_times.mean():.2f} ± {train_times.std():.2f} s ")
    evaluation = {
        "estimator": name,
        "train_time": train_times.mean(),
    }
    evaluation_std = {
        "estimator": name,
        "train_time": train_times.std(),
    }
    for score_name, score_values in scores.items():
        mean_score, std_score = np.mean(score_values), np.std(score_values)
        print(f"{score_name}: {mean_score:.3f} ± {std_score:.3f}")
        evaluation[score_name] = mean_score
        evaluation_std[score_name] = std_score
    evaluations.append(evaluation)
    evaluations_std.append(evaluation_std)

## Feature Extraction using TfidfVectorizer¶

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(
    max_df=0.5,
    min_df=5,
    stop_words="english",
)
t0 = time()
X_tfidf = vectorizer.fit_transform(datasets.data)

print(f"vectorization done in {time() - t0:.3f} s")
print(f"n_samples: {X_tfidf.shape[0]}, n_features: {X_tfidf.shape[1]}")

vectorization done in 5.268 s
n_samples: 10140, n_features: 14779


## After ignoring terms that appear in more than 50% of the documents (as set by max_df=0.5) and terms that are not present in at least 5 documents (set by min_df=5), the resulting number of unique terms n_features is around 14779. We can additionally quantify the sparsity of the X_tfidf matrix as the fraction of non-zero entries divided by the total number of elements.

In [ ]:
print(f"There are {X_tfidf.nnz / np.prod(X_tfidf.shape):.3f}% entries in documents which are non-zero")


There are 0.003% entries in documents which are non-zero
